<a href="https://colab.research.google.com/github/Dhaneshkp/DesktopAssistant/blob/main/DATA_processing_expConv2Dv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

# Set a random seed for reproducibility
np.random.seed(42)

# Parameters
n_samples = 10000
unique_risk_classes = ['A', 'B', 'C', 'D']
n_anomalies = 200
# Generate random ages, durations, and premiums
ages = np.random.randint(18, 70, size=(n_samples, 1))
durations = np.random.randint(1, 20, size=(n_samples, 1))
premiums = ages * durations + np.random.normal(0, 50, size=(n_samples, 1))

# Define risk class multipliers
riskclass_multipliers = {
    'A': 1.05,
    'B': 1.1,
    'C': 1.3,
    'D': 1.0
}

# Initialize the result array
result_array = np.empty((n_samples, 1, 3, len(unique_risk_classes)))

# Iterate over samples and risk classes
for i in range(n_samples):
    for j, risk_class in enumerate(unique_risk_classes):
        # Apply risk class multiplier to premium
        premium_with_risk = premiums[i] * riskclass_multipliers[risk_class]

        # Assign age, duration, and premium to the result array
        result_array[i, 0, :, j] = np.concatenate((ages[i], durations[i], premium_with_risk))

anomaly_indices = np.random.choice(n_samples * len(unique_risk_classes), n_anomalies, replace=False)
for idx in anomaly_indices:
    sample_idx = idx // len(unique_risk_classes)
    risk_class_idx = idx % len(unique_risk_classes)
    result_array[sample_idx, 0, 2, risk_class_idx] += np.random.randint(500, 1000)  # Add a large random value to the premium

print(result_array.shape)  # Output: (10000, 1, 3, 4)
print(result_array)


(10000, 1, 3, 4)
[[[[5.60000000e+01 5.60000000e+01 5.60000000e+01 5.60000000e+01]
   [1.10000000e+01 1.10000000e+01 1.10000000e+01 1.10000000e+01]
   [6.34330506e+02 6.64536721e+02 7.85361579e+02 6.04124292e+02]]]


 [[[6.90000000e+01 6.90000000e+01 6.90000000e+01 6.90000000e+01]
   [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
   [6.28988242e+01 6.58940063e+01 7.78747348e+01 5.99036421e+01]]]


 [[[4.60000000e+01 4.60000000e+01 4.60000000e+01 4.60000000e+01]
   [1.80000000e+01 1.80000000e+01 1.80000000e+01 1.80000000e+01]
   [9.26381249e+02 9.70494642e+02 1.14694821e+03 8.82267856e+02]]]


 ...


 [[[5.70000000e+01 5.70000000e+01 5.70000000e+01 5.70000000e+01]
   [1.80000000e+01 1.80000000e+01 1.80000000e+01 1.80000000e+01]
   [1.08891627e+03 1.14076943e+03 1.34818205e+03 1.03706311e+03]]]


 [[[6.40000000e+01 6.40000000e+01 6.40000000e+01 6.40000000e+01]
   [1.80000000e+01 1.80000000e+01 1.80000000e+01 1.80000000e+01]
   [1.19011274e+03 1.24678478e+03 1.47347292e+03 1

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Conv2DTranspose

X_train, X_test, train_indices, test_indices = train_test_split(
    result_array,
    np.arange(result_array.shape[0]),  # Create indices for the array
    test_size=0.2,
    random_state=42
)
#train_indices, test_indices = train_test_split(data_reshaped.index, test_size=0.2, random_state=42)
# Define the CNN architecture
input_shape = (X_train.shape[1], X_train.shape[2],X_train.shape[3])    # Assuming X_train is a 3D array
input_layer = Input(shape=input_shape)
# Define the CNN architecture
input_shape = (X_train.shape[1], X_train.shape[2],X_train.shape[3])    # Assuming X_train is a 3D array
input_layer = Input(shape=input_shape)

conv_layer = Conv2D(32, (1,3), activation="relu",padding='same')(input_layer)
conv_layer = MaxPooling2D(pool_size=(1,1))(conv_layer)

output_shape = (conv_layer.shape[1], conv_layer.shape[2], conv_layer.shape[3])  # Adjust as needed

# Reshape the output of the convolutional layers
reshape_layer = Reshape(output_shape)(conv_layer)

# Use transposed convolution (Conv2DTranspose) to upsample and reconstruct
deconv_layer = Conv2DTranspose(32, (1, 3), activation='relu', padding='same')(reshape_layer)
output_layer = Conv2D(X_train.shape[3], (1, 1), activation='linear', padding='same')(deconv_layer)  # Output 4 channels

cnn_model = Model(inputs=input_layer, outputs=output_layer)
cnn_model.compile(optimizer='adam', loss='mse')

# ... (rest of your code for training and prediction)

cnn_model = Model(inputs=input_layer, outputs=output_layer)
cnn_model.compile(optimizer='adam', loss='mse')

# Train the model
history = cnn_model.fit(X_train, X_train,
                        epochs=50,
                        batch_size=32,
                        validation_split=0.1,
                        verbose=1)

# Predict the training data
pred_train = cnn_model.predict(X_train)
mse_train = np.mean(np.power(X_train - pred_train,2), axis=1)



Epoch 1/50
225/225 [==============================] - 1s 2ms/step - loss: 13869.4424 - val_loss: 642.9932
Epoch 2/50
225/225 [==============================] - 0s 1ms/step - loss: 326.2699 - val_loss: 242.3096
Epoch 3/50
225/225 [==============================] - 0s 1ms/step - loss: 72.4356 - val_loss: 18.6295
Epoch 4/50
225/225 [==============================] - 0s 2ms/step - loss: 15.2927 - val_loss: 10.6730
Epoch 5/50
225/225 [==============================] - 0s 1ms/step - loss: 9.1747 - val_loss: 5.7004
Epoch 6/50
225/225 [==============================] - 0s 1ms/step - loss: 6.5833 - val_loss: 5.8087
Epoch 7/50
225/225 [==============================] - 0s 1ms/step - loss: 6.5741 - val_loss: 5.8530
Epoch 8/50
225/225 [==============================] - 0s 1ms/step - loss: 4.6090 - val_loss: 4.1395
Epoch 9/50
225/225 [==============================] - 0s 1ms/step - loss: 3.8041 - val_loss: 4.2127
Epoch 10/50
225/225 [==============================] - 0s 1ms/step - loss: 3.4819 - va

In [5]:
pred_train

array([[[[  54.50867  ,   53.961002 ,   53.973083 ,   54.09687  ],
         [   7.8716664,    8.663052 ,    8.76765  ,    8.883755 ],
         [ 385.72867  ,  403.56726  ,  476.2671   ,  366.71426  ]]],


       [[[  34.33031  ,   34.010223 ,   33.96169  ,   34.07517  ],
         [   6.928404 ,    7.4802747,    7.569854 ,    7.601809 ],
         [ 271.97473  ,  284.6656   ,  335.88568  ,  258.59628  ]]],


       [[[  37.748783 ,   38.837173 ,   37.872017 ,   39.578728 ],
         [   8.000684 ,    9.76371  ,   13.768358 ,   12.136143 ],
         [ 445.44443  ,  467.15082  ,  548.1237   , 1253.5728   ]]],


       ...,


       [[[  39.388313 ,   38.961754 ,   38.968235 ,   39.08515  ],
         [   8.865648 ,    9.591866 ,    9.718999 ,    9.763046 ],
         [ 339.05307  ,  354.79645  ,  418.66785  ,  322.33914  ]]],


       [[[  51.288006 ,   51.05699  ,   51.030964 ,   51.07951  ],
         [   3.0776453,    3.302153 ,    3.2157452,    3.3942263],
         [ 138.79434  ,  145.127

In [16]:

# Predict the test data
pred_test = cnn_model.predict(X_test)
mse_test = np.mean(np.power(X_test - pred_test  , 2), axis=1)

# Define a threshold for anomaly detection
threshold = np.percentile(mse_train, 95)

# Identify anomalies in the test set
anomalies = mse_test > threshold

# Print the results
print(f"Threshold: {threshold}")
print(f"Number of anomalies in the test set: {np.sum(anomalies)}")

print(X_test)
print(pred_test)
print(anomalies)

63/63 [==============================] - 0s 1000us/step
Threshold: 2.515566507147924
Number of anomalies in the test set: 1109
[[[[  47.           47.           47.           47.        ]
   [  15.           15.           15.           15.        ]
   [ 675.03702163  707.1816417   835.76012201  642.89240155]]]


 [[[  44.           44.           44.           44.        ]
   [   6.            6.            6.            6.        ]
   [ 174.9418068   183.27236903  216.59461795  166.61124457]]]


 [[[  23.           23.           23.           23.        ]
   [  10.           10.           10.           10.        ]
   [ 733.33356523  222.44468739  262.889176    202.22244308]]]


 ...


 [[[  44.           44.           44.           44.        ]
   [  15.           15.           15.           15.        ]
   [ 680.53920489  712.9458337   842.57234892  648.13257609]]]


 [[[  29.           29.           29.           29.        ]
   [  17.           17.           17.           17.      

In [17]:
# Find indices of anomalies in X_test
anomaly_indices = np.where(anomalies)[0]

# Extract the anomalous records from X_test
anomalous_records = X_test[anomaly_indices]

print("Anomalous Records:")
print(anomalous_records)


Anomalous Records:
[[[[  23.           23.           23.           23.        ]
   [  10.           10.           10.           10.        ]
   [ 733.33356523  222.44468739  262.889176    202.22244308]]]


 [[[  23.           23.           23.           23.        ]
   [  10.           10.           10.           10.        ]
   [ 733.33356523  222.44468739  262.889176    202.22244308]]]


 [[[  23.           23.           23.           23.        ]
   [  10.           10.           10.           10.        ]
   [ 733.33356523  222.44468739  262.889176    202.22244308]]]


 ...


 [[[  50.           50.           50.           50.        ]
   [   8.            8.            8.            8.        ]
   [ 437.09054581  457.90438132  541.15972338 1108.27671029]]]


 [[[  29.           29.           29.           29.        ]
   [  17.           17.           17.           17.        ]
   [ 484.29336466  507.35495346 1121.60130863  461.23177587]]]


 [[[  29.           29.           29.  

In [13]:
# Create a copy of the original test data from the NumPy array
test_indices
anomalies


array([[[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]],

       [[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]],

       [[False, False, False, False],
        [False,  True, False, False],
        [ True,  True, False, False]],

       ...,

       [[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]],

       [[False, False, False, False],
        [False, False, False, False],
        [ True,  True, False, False]],

       [[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]]])

In [ ]:
# prompt: crate an isolation forest to detect anomalies in teh result_df_encoded and fina the index of anomalies in teh data and crate a data frame like test_df_with_anomalies , use the xtrain and xtest etc using the xtrain and xtest indices created in the previuos code, and for isolation forest i think we need y test and train so creat theam , if we drop premium rate then thas sdropiing the ioput

import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

# Assuming 'result_df_encoded' is your DataFrame from the preceding code

# Prepare the data for Isolation Forest
X = result_df_encoded.drop('Premium_rate', axis=1).values  # Assuming 'Premium_rate' is the target variable

# Split the data into training and test sets (using the same indices as before)
X_train = X[train_indices]
X_test = X[test_indices]
y_train = result_df_encoded.loc[train_indices, 'Premium_rate'].values
y_test = result_df_encoded.loc[test_indices, 'Premium_rate'].values

# Create an Isolation Forest model
model = IsolationForest(contamination=0.05, random_state=42)  # Adjust contamination as needed

# Fit the model to the training data
model.fit(X_train)

# Predict anomalies on the test data
y_pred = model.predict(X_test)

# Identify anomalies (-1 indicates an anomaly)
anomalies = y_pred == -1

# Create a copy of the original test DataFrame
test_df_with_anomalies_if = result_df_encoded.iloc[test_indices].copy()

# Add an 'anomaly' column to the test DataFrame
test_df_with_anomalies_if['anomaly'] = anomalies

# Print the results
print(f"Number of anomalies in the test set (Isolation Forest): {np.sum(anomalies)}")

# Display the test DataFrame with anomaly tags
test_df_with_anomalies_if


Number of anomalies in the test set (Isolation Forest): 455


,Age,Duration,Premium_rate,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2,anomaly
21185,22.0,4.0,8.359788,0.0,0.0,1.0,0.0,False
21081,41.0,2.0,4.408889,1.0,0.0,0.0,0.0,False
38072,35.0,9.0,9.813333,0.0,0.0,0.0,1.0,False
37127,21.0,9.0,7.733333,0.0,0.0,0.0,1.0,False
22786,38.0,3.0,5.911330,0.0,0.0,0.0,1.0,False
...,...,...,...,...,...,...,...,...
25543,42.0,3.0,8.059259,0.0,0.0,1.0,0.0,False
23315,28.0,5.0,7.714286,0.0,0.0,1.0,0.0,False
8920,41.0,7.0,12.370370,0.0,0.0,1.0,0.0,False
21822,47.0,5.0,15.828571,0.0,0.0,1.0,0.0,False
